# OPTIMIZACION VERSION 1

En el presente notebook se realizará la optimización de el abastecimiento de ATM´s en los agentes de banca BCP en esta primera version se utilizarán datos simulados
y se empleará el paquete Pyomo para la formulación y resolución del problema de optimización.

In [ ]:
# Optimizacón de recarrga de ATM´S con datos simulados 

In [ ]:
from pyomo.environ import *
import numpy as np

# Datos de ejemplo
dias = ['Lunes', 'Martes', 'Jueves']  # Días disponibles para recargar
dias_totales = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
n_dias = len(dias_totales)

# Datos de la problemática
disponibilidad_recarga = {'Lunes': 100, 'Martes': 120, 'Jueves': 90}  # Disponibilidad máxima para recarga en cada día
demanda = np.array([10, 70, 60, 80, 50, 40, 90, 10, 10, 60, 10, 40, 70, 80])  # Ejemplo de demanda para 14 días
saldo_inicial = 30  # Saldo inicial
capacidad_maxima = 200  # Capacidad máxima

# Crea el modelo de optimización
model = ConcreteModel()

# Variables
model.x = Var(dias, domain=NonNegativeReals)  # Cantidad abastecida en cada día disponible
model.dia_recarga = Var(dias_totales, domain=Binary)  # Día en el que se realiza la recarga (0 o 1)

# Función objetivo: Minimizar el costo
def costo_total(model):
    return sum(0.001 * model.x[d] for d in dias)

model.obj = Objective(rule=costo_total, sense=minimize)

# Restricciones

# Restricción de capacidad
def restriccion_capacidad(model):
    # El saldo inicial más la cantidad abastecida no debe exceder la capacidad máxima
    # Aquí, el saldo más la cantidad abastecida no superará la capacidad máxima del sistema.
    return saldo_inicial + sum(model.x[d] for d in dias) <= capacidad_maxima

model.restriccion_capacidad = Constraint(rule=restriccion_capacidad)

# Restricción de demanda
def demanda_satisfecha(model, dia):
    # Para cada día de recarga, la cantidad abastecida debe ser al menos la demanda.
    if dia in dias:
        idx = dias_totales.index(dia)
        # En lugar de estrictamente igualar la demanda, podemos relajarla un poco
        return model.x[dia] >= demanda[idx] * 0.8  # Ahora, satisface al menos el 80% de la demanda
    else:
        return Constraint.Skip

model.restriccion_demanda = Constraint(dias_totales, rule=demanda_satisfecha)

# Restricción de recarga: Solo se recarga en los días disponibles
def restriccion_dia_recarga(model, dia):
    # Solo se puede recargar en los días disponibles (Lunes, Martes, Jueves)
    if dia in dias:
        return model.dia_recarga[dia] == 1  # Se debe recargar en esos días
    else:
        return model.dia_recarga[dia] == 0  # No recargar en otros días

model.restriccion_dia_recarga = Constraint(dias_totales, rule=restriccion_dia_recarga)

# Restricción sobre la cantidad de recarga disponible en cada día
def restriccion_disponibilidad_recarga(model, dia):
    if dia in dias:
        return model.x[dia] <= disponibilidad_recarga[dia]  # No recargar más de lo disponible
    else:
        return Constraint.Skip

model.restriccion_disponibilidad_recarga = Constraint(dias_totales, rule=restriccion_disponibilidad_recarga)

# Resolver el modelo
solver = SolverFactory('glpk')
solver.solve(model, tee=True)

# Resultados




GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write C:\Users\Arthur\AppData\Local\Temp\tmpz9m5smte.glpk.raw --wglp C:\Users\Arthur\AppData\Local\Temp\tmp900k8euo.glpk.glp
 --cpxlp C:\Users\Arthur\AppData\Local\Temp\tmpfza4izc4.pyomo.lp
Reading problem data from 'C:\Users\Arthur\AppData\Local\Temp\tmpfza4izc4.pyomo.lp'...
C:\Users\Arthur\AppData\Local\Temp\tmpfza4izc4.pyomo.lp:81: warning: lower bound of variable 'x10' redefined
C:\Users\Arthur\AppData\Local\Temp\tmpfza4izc4.pyomo.lp:81: warning: upper bound of variable 'x10' redefined
14 rows, 10 columns, 16 non-zeros
7 integer variables, all of which are binary
88 lines were read
Writing problem data to 'C:\Users\Arthur\AppData\Local\Temp\tmp900k8euo.glpk.glp'...
59 lines were written
GLPK Integer Optimizer, v4.65
14 rows, 10 columns, 16 non-zeros
7 integer variables, all of which are binary
Preprocessing...
1 row, 3 columns, 3 non-zeros
0 integer variables, none of which are binary
Scaling...
 A: mi

## Conclusion:


In [2]:
# Mostrar la cantidad abastecida y qué días se recargan
print("Cantidad abastecida por día y si se recarga ese día:")
for d in dias:
    cantidad = model.x[d].value
    recarga = "Sí" if cantidad > 0 else "No"
    print(f"{d}: Cantidad abastecida = {cantidad:.2f}, Recarga: {recarga}")

Cantidad abastecida por día y si se recarga ese día:
Lunes: Cantidad abastecida = 8.00, Recarga: Sí
Martes: Cantidad abastecida = 56.00, Recarga: Sí
Jueves: Cantidad abastecida = 64.00, Recarga: Sí


Se obtuvo como resultado optimo la recarga de los 3 días de semana 